In [45]:
##import relevant packages and apply settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date as dt
from scipy.stats import chi2_contingency
import scipy.stats as stats
import researchpy as rp
import requests
import xml.etree.ElementTree as et
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)

In [21]:
#read in prepared file (i.e. the cleaned and prepped skills test data set)
file ='prepared.csv'
clients = pd.read_csv(file)

First I wanted to take a look at the average days since last login and last field visit, as well as taking a second look at funds raised and unique donors.

In [22]:
#set 9999 and 01/01/1990 as missing
clients = clients.replace(9999, np.nan)
clients = clients.replace(1/1/1900, np.nan)
clients = clients.replace('1/1/1900 0:00', np.nan)
#clients = clients.dropna()

#calculate average days since last login and dayts since last field visit, aggregated by status
print(clients.groupby('status')['last_login_date_lapse'].mean(), '\n', clients.groupby('status')['most_recent_field_visit_date_lapse'].mean())

#calculate average days since funds and donors, aggregated by status
print(clients.groupby('status')['all_time_funds_raised'].mean(), '\n', clients.groupby('status')['all_time_unique_donors'].mean())

status
Delinquent - Due Diligence    415.114441
Emerging                      569.445650
Leader                        367.865779
Partner                       271.794500
Superstar                     376.843832
Name: last_login_date_lapse, dtype: float64 
 status
Delinquent - Due Diligence    40541.311681
Emerging                      44332.082412
Leader                        28391.517418
Partner                       37260.286491
Superstar                     24815.994751
Name: most_recent_field_visit_date_lapse, dtype: float64
status
Delinquent - Due Diligence      7415.243963
Emerging                        4923.308919
Leader                         80815.486855
Partner                        39021.637446
Superstar                     108426.732415
Name: all_time_funds_raised, dtype: float64 
 status
Delinquent - Due Diligence     21.989354
Emerging                        3.895473
Leader                        304.799180
Partner                        73.766697
Superstar          

In [16]:
clients.head(10)

,Unnamed: 0,name_x,status,country_of_registration,year_founded,original_vetting_approval_date,most_recent_vetting_approval_date,vetting_expiration_date,approx_annual_budget,number_of_currently_active_projects,number_of_funded_projects,number_of_retired_projects,earliest_project_created_date,most_recent_project_created_date,earliest_project_deactivation_date,most_recent_project_deactivation_date,last_login_date,completed_accelerator_program,number_of_field_visits,most_recent_field_visit_date,all_time_funds_raised,all_time_unique_donors,acquisition_source_category,all_time_unque_project_themes,active_contacts,name_y,region,sub-region,alpha-3,Alpha-3 code,Latitude (average),Longitude (average),Animal Welfare,Child Protection,Climate Action,Peace and Reconciliation,Disaster Recovery,Economic Growth,Education,Ecosystem Restoration,Gender Equality,Physical Health,End Human Trafficking,Justice and Human Rights,Sport,Digital Literacy,Food Security,Arts and Culture,LGBTQIA+ Equality,COVID-19,Clean Water,Disability Rights,End Abuse,Mental Health,Racial Justice,Refugee Rights,Reproductive Health,Safe Housing,Sustainable Agriculture,Wildlife Conservation,regions,original_vetting_approval_date_year,original_vetting_approval_date_month,original_vetting_approval_date_delta,most_recent_vetting_approval_date_year,most_recent_vetting_approval_date_month,most_recent_vetting_approval_date_delta,vetting_expiration_date_year,vetting_expiration_date_month,vetting_expiration_date_delta,most_recent_field_visit_date_year,most_recent_field_visit_date_month,most_recent_field_visit_date_delta,earliest_project_created_date_year,earliest_project_created_date_month,earliest_project_created_date_delta,most_recent_project_created_date_year,most_recent_project_created_date_month,most_recent_project_created_date_delta,earliest_project_deactivation_date_year,earliest_project_deactivation_date_month,earliest_project_deactivation_date_delta,most_recent_project_deactivation_date_year,most_recent_project_deactivation_date_month,most_recent_project_deactivation_date_delta,last_login_date_year,last_login_date_month,last_login_date_delta,last_login_date_lapse,original_vetting_approval_date_lapse,most_recent_field_visit_date_lapse,earliest_project_created_date_lapse
1537,1537.0,Birthing Kit Foundation (Australia),Leader,Australia,2006.0,12/21/2010,10/9/2019,10/9/2021,583933.0,1.0,0.0,0.0,12/15/2010 20:47,2/2/2020 21:16,11/12/2012 14:09,4/21/2020 12:17,8/24/2020 3:04,0,1.0,7/14/2015,30055.34,471,word_of_mouth,"Child Protection,Economic Growth,Gender Equali...",4,Australia,Oceania,Australia and New Zealand,AUS,AUS,-27.0,133.0,False,True,False,False,False,True,False,False,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Oceania,2010,12,3564,2019,10,350,2021,10,-381,2015,7,1898,2010,12,3569,2020,2,233,2012,11,2871,2020,4,154,2020,8,29,410,3945,2279,3950
1720,1720.0,Australia Zoo Wildlife Warriors,Leader,Australia,2002.0,2/28/2011,9/18/2019,9/18/2021,1692980.0,1.0,0.0,1.0,3/1/2011 18:26,10/1/2014 20:21,1/5/2015 11:09,1/5/2015 11:09,8/24/2020 18:43,0,1.0,4/5/2018,316140.42,1045,blank,"Animal Welfare,Ecosystem Restoration",3,Australia,Oceania,Australia and New Zealand,AUS,AUS,-27.0,133.0,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Oceania,2011,2,3495,2019,9,371,2021,9,-360,2018,4,902,2011,3,3493,2014,10,2183,2015,1,2087,2015,1,2087,2020,8,29,389,3855,1262,3853
2151,2151.0,The Sunflower Foundation (Australia) Inc.,Partner,Australia,2008.0,10/20/2016,3/27/2019,3/27/2021,12949.0,3.0,1.0,0.0,11/20/2016 22:27,4/20/2018 8:17,3/21/2018 9:53,3/21/2018 9:53,8/25/2020 10:45,1,1.0,5/6/2019,13084.07,153,google_ads,"Child Protection,Economic Growth,Education,Gen...",3,Australia,Oceania,Australia and New Zealand,AUS,AUS,-27.0,133.0,False,True,False,False,False,True,True,False,True,True,False,True,False,Fal

Here I'm taking a first look at the bivariate relationships between some of the cateogorical variables of interest and status using a series of crosstabs and chi-square tests.

In [16]:
#crosstabs and chisquare tests for categorical variables
#categorical variables of interest

cats_all = ['status', 'region', 'completed_accelerator_program', 'Animal Welfare', 'Child Protection',
       'Climate Action', 'Peace and Reconciliation', 'Disaster Recovery',
       'Economic Growth', 'Education', 'Ecosystem Restoration',
       'Gender Equality', 'Physical Health', 'End Human Trafficking',
       'Justice and Human Rights', 'Sport', 'Digital Literacy',
       'Food Security', 'Arts and Culture', 'LGBTQIA+ Equality', 'COVID-19',
       'Clean Water', 'Disability Rights', 'End Abuse', 'Mental Health',
       'Racial Justice', 'Refugee Rights', 'Reproductive Health',
       'Safe Housing', 'Sustainable Agriculture', 'Wildlife Conservation' ]
cats = ['region', 'completed_accelerator_program', 'Animal Welfare', 'Child Protection',
       'Climate Action', 'Peace and Reconciliation', 'Disaster Recovery',
       'Economic Growth', 'Education', 'Ecosystem Restoration',
       'Gender Equality', 'Physical Health', 'End Human Trafficking',
       'Justice and Human Rights', 'Sport', 'Digital Literacy',
       'Food Security', 'Arts and Culture', 'LGBTQIA+ Equality', 'COVID-19',
       'Clean Water', 'Disability Rights', 'End Abuse', 'Mental Health',
       'Racial Justice', 'Refugee Rights', 'Reproductive Health',
       'Safe Housing', 'Sustainable Agriculture', 'Wildlife Conservation']

cats_df = clients[cats_all]
a = 0.5

for var in cats:
    crosstab = pd.crosstab(cats_df['status'], cats_df[var])
    stat, p, dof, expected = chi2_contingency(crosstab)
    print(crosstab, '\n\n', pd.crosstab(cats_df['status'], cats_df[var], normalize = 'index'), '\n\n', pd.crosstab(cats_df['status'], cats_df[var], normalize = 'columns'), '\n\n', pd.crosstab(cats_df['status'], cats_df[var]), '\n\n p: ', p, p < a)
    

region                      Africa  Americas  Asia  Europe  Oceania
status                                                             
Delinquent - Due Diligence    2438      4883  1628    1102       81
Emerging                      1700      2055   606     407       25
Leader                         140       481   186     161        8
Partner                        288      1683   527     751       60
Superstar                      135       333   161     124        9 

 region                        Africa  Americas      Asia    Europe   Oceania
status                                                                      
Delinquent - Due Diligence  0.240624  0.481938  0.160679  0.108764  0.007994
Emerging                    0.354684  0.428750  0.126434  0.084916  0.005216
Leader                      0.143443  0.492828  0.190574  0.164959  0.008197
Partner                     0.087035  0.508613  0.159263  0.226957  0.018132
Superstar                   0.177165  0.437008  0.211286  0

Superstar                   0.036324  0.040811 

 Economic Growth             False  True 
status                                  
Delinquent - Due Diligence   5869   4276
Emerging                     2753   2040
Leader                        616    360
Partner                      2276   1033
Superstar                     434    328 

 p:  2.162460738903548e-30 True
Education                   False  True 
status                                  
Delinquent - Due Diligence   2703   7442
Emerging                     1230   3563
Leader                        240    736
Partner                       964   2345
Superstar                     139    623 

 Education                      False     True 
status                                        
Delinquent - Due Diligence  0.266437  0.733563
Emerging                    0.256624  0.743376
Leader                      0.245902  0.754098
Partner                     0.291327  0.708673
Superstar                   0.182415  0.817585 

 Educati

Digital Literacy            False  True 
status                                  
Delinquent - Due Diligence   8159   1986
Emerging                     4070    723
Leader                        863    113
Partner                      2800    509
Superstar                     661    101 

 Digital Literacy               False     True 
status                                        
Delinquent - Due Diligence  0.804239  0.195761
Emerging                    0.849155  0.150845
Leader                      0.884221  0.115779
Partner                     0.846177  0.153823
Superstar                   0.867454  0.132546 

 Digital Literacy               False     True 
status                                        
Delinquent - Due Diligence  0.492902  0.578671
Emerging                    0.245877  0.210664
Leader                      0.052136  0.032925
Partner                     0.169154  0.148310
Superstar                   0.039932  0.029429 

 Digital Literacy            False  True 
statu

Superstar                     759      3 

 p:  4.312108046377436e-81 True
Mental Health               False  True 
status                                  
Delinquent - Due Diligence   9513    632
Emerging                     4767     26
Leader                        972      4
Partner                      3227     82
Superstar                     760      2 

 Mental Health                  False     True 
status                                        
Delinquent - Due Diligence  0.937703  0.062297
Emerging                    0.994575  0.005425
Leader                      0.995902  0.004098
Partner                     0.975219  0.024781
Superstar                   0.997375  0.002625 

 Mental Health                  False     True 
status                                        
Delinquent - Due Diligence  0.494464  0.847185
Emerging                    0.247778  0.034853
Leader                      0.050522  0.005362
Partner                     0.167732  0.109920
Superstar            

Some initial thoughts here--it looks like organizations from any regions about about as likely to be Delinquent (though it doesn't mean they're Delinquent for the same reasons). It also looks like we've picked up our efforts in Africa lately, given the high proportion of Emerging organizations (~36%). Numbers-wise, most organizations are still coming from the Americas, though, in every category.

As you might expect, not many of the Delinquent organizations finished the Accelerator Program, but without more information about the program (when it began, who it targets, etc.), it's difficult to say what the ultimate impact of that program might be.

The thematic categories need a bit more processing in order to draw meaningful conclusions, but that's a first step in processing.

The next step I took was reviewing the numerical variables that I believe could have an impact on status through a series of one-way ANOVAs. Ideally, I'd like to do a bit more assumptions testing, reviewing variable distributions and post-hoc testing here, but this is as far as I could get this time around.

In [70]:
#ANOVAs for numerical variables
#numerical variables of interest

nums_all = ['status', 'approx_annual_budget', 'number_of_currently_active_projects', 'number_of_funded_projects', 'number_of_field_visits',
           'all_time_funds_raised', 'all_time_unique_donors', 'active_contacts', 'original_vetting_approval_date_year', 'earliest_project_created_date_year']
nums = ['approx_annual_budget', 'number_of_currently_active_projects', 'number_of_funded_projects', 'number_of_field_visits',
           'all_time_funds_raised', 'all_time_unique_donors', 'original_vetting_approval_date_year', 'earliest_project_created_date_year']

num_df = clients[nums_all]

#replace values with NaN
#set 9999 and 01/01/1990 as missing
num_df = num_df.replace(9999, np.nan)
num_df = num_df.replace(1900, np.nan)
num_df = num_df.replace('1/1/1900', np.nan)
num_df = num_df.replace('1/1/1900 0:00', np.nan)

a = 0.05




for var in nums:
    df = num_df[[var, 'status']].dropna()
    print(var, '\n\n', rp.summary_cont(df[var].groupby(df['status'])))
    print(stats.f_oneway(df[var][df['status'] == 'Delinquent - Due Diligence'], 
                         df[var][df['status'] == 'Emerging'],
                         df[var][df['status'] == 'Partner'],
                         df[var][df['status'] == 'Leader'],
                         df[var][df['status'] == 'Superstar']))
    

        




approx_annual_budget 

                                 N          Mean            SD            SE  \
status                                                                        
Delinquent - Due Diligence  10142  1.264719e+07  2.560249e+08  2.542263e+06   
Emerging                     4791  8.071908e+06  3.223618e+08  4.657260e+06   
Leader                        976  1.196867e+07  1.283703e+08  4.109033e+06   
Partner                      3309  1.245264e+08  4.867346e+09  8.461431e+07   
Superstar                     762  3.059829e+06  2.501625e+07  9.062430e+05   

                               95% Conf.      Interval  
status                                                  
Delinquent - Due Diligence  7.663853e+06  1.763053e+07  
Emerging                   -1.058460e+06  1.720228e+07  
Leader                      3.905106e+06  2.003224e+07  
Partner                    -4.137526e+07  2.904281e+08  
Superstar                   1.280796e+06  4.838862e+06  
F_onewayResult(statis

Generally, this confirms what I discussed before in reviewing the distributions in the data preparation notebook, but this applies a statistical test comparing the means of each of the groups--there is some small difference in approximate annual budget, Delinquent organizations have far fewer currently active projects as well as less funded projects, though they are nearly on par with Partners here.

The number of field visits is unrealiable here because there are too few values for Emerging clients, so they'd need to be removed to see if any impact emerges, though my hunch is that it wouldn't, given the similar means of the other three groups. Again, there is clear difference between groups in terms of funds raised and number of donors.

Overall, it looks like Delinquent clients had on average an earlier original vetting date and first project date, suggesting either that long-term organizations are more likely to become Delinquent or that delinquency was an issue experienced early on in Global Giving's history and it may not have been as significant an issue in more recent years.